In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()
# Import packages
from pyspark.sql import SparkSession
import time
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_df.show()












Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [44.7 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,889 kB]
Get:13 http://archive.ubuntu.c

In [3]:
!echo $SPARK_HOME

/content/spark-3.4.2-bin-hadoop3


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [28]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView("home_sales")


In [29]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import year, avg, format_number

four_bedroom_homes = home_df.filter(home_df.bedrooms == "4")

average_price_per_year = four_bedroom_homes.withColumn("year", year(home_df.date)) \
                                           .groupBy("year") \
                                           .agg(format_number(avg("price"), 2).alias("average_price"))

average_price_per_year.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|   296,363.88|
|2019|   300,263.70|
|2020|   298,353.78|
|2021|   301,819.44|
+----+-------------+



In [30]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_bath_homes = home_df.filter((home_df.bedrooms == "3") & (home_df.bathrooms == "3"))

average_price_by_build_year = three_bed_bath_homes.groupBy("date_built") \
                                                  .agg(format_number(avg("price"), 2).alias("average_price"))


average_price_by_build_year.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|   290,555.07|
|      2012|   293,683.19|
|      2017|   292,676.79|
|      2014|   290,852.27|
|      2013|   295,962.27|
|      2011|   291,117.47|
|      2015|   288,770.30|
|      2010|   292,859.62|
+----------+-------------+



In [31]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
target_homes = home_df.filter(
    (home_df.bedrooms == "3") &
    (home_df.bathrooms == "3") &
    (home_df.floors == "2") &
    (home_df.sqft_living >= 2000)
)
average_price_by_build_year = target_homes.groupBy("date_built") \
                                          .agg(format_number(avg("price"), 2).alias("average_price"))
average_price_by_build_year.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|   293,965.10|
|      2012|   307,539.97|
|      2017|   280,317.58|
|      2014|   298,264.72|
|      2013|   303,676.79|
|      2011|   276,553.81|
|      2015|   297,609.97|
|      2010|   285,010.22|
+----------+-------------+



In [38]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
from pyspark.sql.functions import col, format_number, avg
start_time = time.time()
average_price_by_view = home_df.groupBy("view") \
                               .agg(format_number(avg("price"), 2).alias("average_price")) \
                               .filter(col("average_price") >= 350000) \
                               .orderBy(col("view").desc())




print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06043505668640137 seconds ---


In [33]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [34]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [37]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

average_price_by_view_cached = spark.sql("""
    SELECT view, format_number(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")




print("--- %s seconds ---" % (time.time() - start_time))


--- 0.038794517517089844 seconds ---


In [39]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
path_to_save = "/path/to/save/parquet/home_sales_partitioned_by_date_built"
home_df.write.partitionBy("date_built").parquet(path_to_save)


In [40]:
# 11. Read the parquet formatted data.
path_to_read = "/path/to/save/parquet/home_sales_partitioned_by_date_built"
parquet_df = spark.read.parquet(path_to_read)
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [41]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [43]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query_result = spark.sql("""
    SELECT view, format_number(AVG(price), 2) AS average_price
    FROM parquet_home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.037499427795410156 seconds ---


In [46]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [47]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
